In [2]:
import pandas as pd
import os
from pyzotero import zotero

In [3]:
ZOTERO_KEY = os.environ.get('ZOTERO_KEY')
ZOTERO_USER= os.environ.get('ZOTERO_USER')
zot = zotero.Zotero(ZOTERO_USER, 'user', ZOTERO_KEY)

In [4]:
df = pd.read_excel('data\\AfO_FullBiblio.xlsx')
# drop cells with "s.a." and "s." in Author
df = df[df["Autor"].str.contains(" s.") == False]
#19709 rows
df = df.set_index("ID")

In [5]:
#split author into first and last name
df['autor_first'] = "-"
df['autor_last'] = "-"

for i, row in df.iterrows():
    if not '–' in row['Autor'] and not '(Hrsg.)' in row['Autor']:
        row['autor_last'] = row['Autor'].split(',')[0]
        try:
            row['autor_first'] = row['Autor'].split(', ')[1]
        except:
            continue

In [7]:
for i, row in df.iterrows():
    if not type(row['Zitat']) is str:
        template = zot.item_template('book')
        template['title'] = row['Titel']
                
        if type(row['Zitat']) is str:
            template['shortTitle'] = row['Abkürzung']  
        
        try:
             template['place'] = row['Buchdaten'].split(',')[0]
        except:
            continue

        try:
            template['publisher'] = row['Buchdaten'].split(', ')[1]
        except:
            continue
            
        try:
            y=int(row['Buchdaten'].split('(')[0].split(", ")[-1][1:3])
            if y > 25:
                template['date'] = '19'+str(y)
            else:
                template['date'] = '20'+str(y).zfill(2)
        except:
            continue
        
        if 'Hrsg.' not in row['Autor']:
            if not row['Autor']: 
                   continue
            else:    
                template['creators'][0]['lastName'] = row['autor_last']
            try:
                template['creators'][0]['firstName'] = row['autor_first'][:-1]
            except:
                continue
        else:
            continue

        resp = zot.create_items([template])
         #print(template)

KeyboardInterrupt: 

In [ ]:
#create list of abbreviations
abbreviations = []
for i, row in df.iterrows():
    if type(row['Abkürzung']) is str:
        abbreviations.append(row['Abkürzung'])

In [ ]:
for i, row in df.iterrows():
    if type(row['Zitat']) is str and any(ele in row['Zitat'] for ele in abbreviations):
        template = zot.item_template('bookSection')
        
        template['title'] = row['Titel'].rstrip(':')
        template['shortTitle'] = row['Zitat']
        
        try:
            x = row['Zitat'].split('p. ')[1].split(' ')[0].rstrip('.')
            template['pages'] = x
        except:
            continue
        
        if not row['Autor']: 
            continue
        else:    
            template['creators'][0]['lastName'] = row['autor_last']
        try:
            template['creators'][0]['firstName'] = row['autor_first'][:-1]
        except:
            continue
        
        resp = zot.create_items([template])
    
    else:
        template = zot.item_template('journalArticle')
        
        template['title'] = row['Titel']
        template['shortTitle'] = row['Zitat']
        
        try:
            x = ''.join(row['Zitat'].split('p. ')[1]).split(' ')[0].rstrip('.')
            template['pages'] = x
        except:
            continue
        
        if not row['Autor']: 
               continue
        else:    
            template['creators'][0]['lastName'] = row['autor_last']
        try:
            template['creators'][0]['firstName'] = row['autor_first'][:-1]
        except:
            continue
        
        resp = zot.create_items([template])